# Audio Separator

Takes raw video files, isolates the audio channels and saves them in various formats
* ch0 isolated
* ch1 isolated
* ch0 inverted + channel 1 added
* ch1 inverted + channel 0 added

In [ ]:
import moviepy.editor as mp
import wave
import numpy as np

from utils import io_utils

import os

### Extract and isolate audio tracks from video

In [ ]:
proj_tree = io_utils.ProjectTree('../projects')

for project in proj_tree.get_projects():
    video_src = project.get_media('video/mov/raw')
    audio_dst = project.make_media('audio/wav')
        
    for video_src_path, file_name in zip(video_src.get_file_paths(), video_src.get_file_names()):
        print('Extracting audio from ' + video_src_path + '...')
        
        clip = mp.VideoFileClip(video_src_path)
        clip.audio.write_audiofile(audio_dst.get_root() + '/' + file_name + '.wav', codec='pcm_s16le')
        
        # channel separation code from
        # https://stackoverflow.com/questions/51275725/how-do-you-separate-each-channel-of-a-two-channel-wav-file-into-two-different-fi
        wav = wave.open(audio_dst.get_root() + '/' + file_name + '.wav', 'r')
        nch = wav.getnchannels()
        depth = wav.getsampwidth()
        wav.setpos(0)
        sdata = wav.readframes(wav.getnframes())

        typ = { 1: np.int8, 2: np.int16, 4: np.int32 }.get(depth)
        if not typ:
            raise ValueError("sample width {} not supported".format(depth))
            
        data = np.frombuffer(sdata, dtype=typ)
        channels_data = [data[raw_channel::nch] for raw_channel in range(nch)]
        raw_audio_channels_dst = [audio_dst.make_media('raw/ch{}'.format(raw_channel)) for raw_channel in range(nch)]
        
        for channel, (channel_data, raw_audio_channel_dst) in enumerate(zip(channels_data, raw_audio_channels_dst)):
            print("\tExtracting channel {} out of {} channels, {}-bit depth".format(channel+1, nch, depth*8))
            
            outwav = wave.open(raw_audio_channel_dst.get_root() + '/' + file_name + '.wav', 'w')
            outwav.setparams(wav.getparams())
            outwav.setnchannels(1)
            outwav.writeframes(channel_data.tostring())
            outwav.close()
            
        #mixing channels
        mix_audio_channels_dst = [audio_dst.make_media(mix_channel) for mix_channel in ['mix/ch0_inv_ch1_added', 'mix/ch1_inv_ch0_added']]
        
        for inv_channel, (channel_data, mix_audio_channel_dst) in enumerate(zip(channels_data, mix_audio_channels_dst)):
            print("\tMixing inverted channel {} and channel {}".format(inv_channel, (inv_channel+1)%2))
                  
            mixed_audio = np.add((-1)*channels_data[inv_channel], channels_data[(inv_channel+1)%2])
            
            outwav = wave.open(mix_audio_channel_dst.get_root() + '/' + file_name + '.wav', 'w')
            outwav.setparams(wav.getparams())
            outwav.setnchannels(1)
            outwav.writeframes(mixed_audio.tostring())
            outwav.close()
            
        #remove extracted audio, only save separated channels
        os.remove(audio_dst.get_root() + '/' + file_name + '.wav')